<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train4_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd

In [183]:

train4 = pd.read_csv(r'/content/train_data4.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [184]:

train4['Target'] = train4['Target'].replace(['Others'],'Invalid')


In [185]:
train4['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [186]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [187]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train4_Xtfidf = tfidf_vect.fit_transform(train4['Sentence'])


In [188]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [189]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [190]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [191]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [192]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [193]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [194]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train4_Xtfidf,train4['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6902654867256637
                  precision    recall  f1-score   support

        Analysis       0.53      0.40      0.46       127
      Conclusion       0.77      0.57      0.66        42
           Facts       0.74      0.92      0.82       487
         Invalid       0.54      0.46      0.50       128
           Issue       0.61      0.57      0.59        49
Rule/Law/Holding       0.76      0.23      0.35        71

        accuracy                           0.69       904
       macro avg       0.66      0.52      0.56       904
    weighted avg       0.68      0.69      0.67       904



In [195]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [196]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 4, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2]
[0.9976446446390893, 0.9998857057086342, 0.9801851537939786, 0.9996778704598464, 0.9917548188676435, 0.9972964344657995, 0.9882755078528963, 0.9980808833294768, 0.9964582876583572, 0.9986752612988802, 0.9952846433771747, 0.9881233361491617, 0.998452955792544, 0.9902520511538685, 0.9989298859191083, 0.9919845619422457, 0.9995801200501845, 0.999903420960482, 0.9863593943567419, 0.9955590047301007, 0.9958672316262658, 0.9848807492130575, 0.99955780235281, 0.9975122807382744, 0.9967036216499021, 0.9845105171262126, 0.990860845230983, 0.9887112230813264, 0.9990319482287313, 0.9927192284751014, 0.9802892613725194, 0.9886457263076629, 0.9935631140956862, 0.9937806645584201, 0.9997870816235, 0.9942379627937862, 0.9985296562277255, 0.9931592302229569, 0.9810638890281912]
[4, 5, 6, 11, 12, 18, 21, 23, 24, 29, 31, 34, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 56, 60, 62, 69, 70, 74,

In [197]:
unlabel_1=unlabel_1.loc[[4, 5, 6, 11, 12, 18, 21, 23, 24, 29, 31, 34, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 56, 60, 62, 69, 70, 74, 76, 77, 79, 85, 87, 89, 94, 95, 96, 98, 100],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts
11,1604,police notified deceased found lying mrs moers...,Facts
12,1102,shown vine street drug storeâs supply narcotic...,Facts


In [198]:
frame_1 = [train4,unlabel_1]
train4_1 = pd.concat(frame_1)
len(train4_1)

3515

In [199]:
x_train_1 = tfidf_vect.transform(train4_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train4_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6685633001422475
                  precision    recall  f1-score   support

        Analysis       0.48      0.44      0.46       103
      Conclusion       0.71      0.57      0.63        35
           Facts       0.75      0.90      0.82       351
         Invalid       0.53      0.48      0.50       113
           Issue       0.59      0.56      0.57        36
Rule/Law/Holding       0.58      0.22      0.31        65

        accuracy                           0.67       703
       macro avg       0.61      0.53      0.55       703
    weighted avg       0.65      0.67      0.65       703



In [200]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9966675320257719, 0.9988668585794315, 0.9847598558249304, 0.9865961805586067, 0.9955920755920773, 0.9999545533318023, 0.9880845631847646, 0.9948830570347841, 0.9954402113503806, 0.9984733008192713, 0.9831298859240253, 0.983588650747166, 0.996516392750617, 0.9998666550031514, 0.9992253788315488, 0.9998151096729316, 0.9952146367327604, 0.9995264028870234, 0.9965332093057305, 0.9994237470846945, 0.9822207984472889, 0.9988263637935259, 0.9998029155723939, 0.9986017691663546, 0.9969230408237584, 0.989428203243213, 0.9848429950240728, 0.9974546207765723, 0.984571816151361, 0.9976371877097562, 0.9966563577037797, 0.995590109745926, 0.998594494061772, 0.9973729362198374, 0.983102827084067, 0.9943871324709839]
[0, 2, 4, 7, 8, 12, 15, 20, 21, 29, 32, 34, 36, 37, 39, 42, 44, 49, 55, 59, 60, 63, 64, 69, 70, 71, 74, 75, 77, 83, 85, 86, 88, 91, 95, 99]
36
36
36


In [201]:
unlabel_2 = unlabel_2.loc[[0, 2, 4, 7, 8, 12, 15, 20, 21, 29, 32, 34, 36, 37, 39, 42, 44, 49, 55, 59, 60, 63, 64, 69, 70, 71, 74, 75, 77, 83, 85, 86, 88, 91, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
7,3150,finding door facing street seen man walking pr...,Facts
8,710,appellant indicted robbery indictment alleging...,Analysis


In [202]:
frame_2 = [train4_1,unlabel_2]
train4_2 = pd.concat(frame_2)
len(train4_2)

3551

In [203]:
x_train_2 = tfidf_vect.transform(train4_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train4_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6423357664233577
                  precision    recall  f1-score   support

        Analysis       0.49      0.38      0.43       143
      Conclusion       0.67      0.31      0.42        45
           Facts       0.71      0.90      0.80       485
         Invalid       0.48      0.43      0.46       164
           Issue       0.64      0.33      0.44        63
Rule/Law/Holding       0.56      0.32      0.41        59

        accuracy                           0.64       959
       macro avg       0.59      0.45      0.49       959
    weighted avg       0.62      0.64      0.62       959



In [204]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9943823182110042, 0.9926367821142688, 0.9995850635196825, 0.9993186700991858, 0.999471649572117, 0.9996898574637751, 0.9854427457649629, 0.9950900444367399, 0.9983312842271318, 0.9922339970907735, 0.9974898206168884, 0.9989180819543801, 0.9979641054448877, 0.9975304144523933, 0.9943549832715738, 0.9990288444040588, 0.9908802701749955, 0.9954513072111816, 0.996989392204631, 0.9929889502674033, 0.9955277809363239, 0.9967761716987866, 0.9998492919208802, 0.9913438653843789, 0.9997109625115106, 0.9969295374411717, 0.9958394039600512, 0.9994068942755336, 0.9964480378810743, 0.9804612891443031, 0.9994698916597736, 0.9843891087257272, 0.9980552752039418, 0.9934176158707733, 0.998147983441557, 0.988120699276585]
[0, 3, 7, 8, 12, 20, 24, 26, 27, 28, 29, 30, 35, 36, 40, 43, 46, 47, 52, 53, 54, 57, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 79, 89, 96]
36
36
36


In [206]:
unlabel_3=unlabel_3.loc[[0, 3, 7, 8, 12, 20, 24, 26, 27, 28, 29, 30, 35, 36, 40, 43, 46, 47, 52, 53, 54, 57, 63, 64, 65, 66, 67, 68, 70, 71, 72, 74, 75, 79, 89, 96],:]
frame_3 = [train4_2,unlabel_3]
train4_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train4_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train4_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.692200557103064
                  precision    recall  f1-score   support

        Analysis       0.49      0.46      0.47        90
      Conclusion       0.85      0.47      0.61        49
           Facts       0.77      0.90      0.83       382
         Invalid       0.52      0.51      0.51       102
           Issue       0.70      0.42      0.53        45
Rule/Law/Holding       0.51      0.36      0.42        50

        accuracy                           0.69       718
       macro avg       0.64      0.52      0.56       718
    weighted avg       0.68      0.69      0.68       718



In [207]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 5, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2]
[0.9980178849698841, 0.9989771142049338, 0.9871592337941746, 0.9913776987466426, 0.9998163477829493, 0.9991454646015584, 0.9835342069107896, 0.9994741309531463, 0.9976431711381895, 0.9985121030721928, 0.9982248335352013, 0.9965857303238167, 0.9995187191577246, 0.9963216721953307, 0.9955232055062226, 0.99904937737463, 0.9992406156453458, 0.9944290060292313, 0.9980576427387633, 0.9993293728781075, 0.9913505516047493, 0.9946140092067212, 0.9983786023578985, 0.9936720105297967, 0.9986924525287639, 0.9859577489677742, 0.981443345875114, 0.9972022562293177, 0.9810899354817582, 0.9993587960002012, 0.9984090353381457, 0.9999652605377016, 0.9923232745369649, 0.9978062387443262, 0.9941286867698214, 0.9819011723686496, 0.9982337845095024, 0.9853499520497154, 0.9880469779341438, 0.9921447161609422]
[2, 4, 6, 7, 10, 15, 16, 18, 21, 22, 27, 28, 29, 31, 32, 34, 37, 38, 39, 40, 46, 

In [208]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 10, 15, 16, 18, 21, 22, 27, 28, 29, 31, 32, 34, 37, 38, 39, 40, 46, 49, 50, 55, 56, 60, 64, 67, 68, 69, 70, 75, 80, 85, 87, 89, 93, 94, 97, 99 ],:]

In [209]:
frame_4 = [train4_3,unlabel_4]
train4_4 = pd.concat(frame_4)
len(train4_4)
x_train_4 = tfidf_vect.transform(train4_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train4_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6704081632653062
                  precision    recall  f1-score   support

        Analysis       0.48      0.41      0.44       146
      Conclusion       0.78      0.54      0.64        52
           Facts       0.78      0.87      0.82       516
         Invalid       0.46      0.49      0.47       136
           Issue       0.57      0.43      0.49        63
Rule/Law/Holding       0.52      0.42      0.46        67

        accuracy                           0.67       980
       macro avg       0.60      0.53      0.55       980
    weighted avg       0.66      0.67      0.66       980



In [210]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0]
[0.9989189451382754, 0.997219674960325, 0.9957307086263015, 0.9941135107323981, 0.996329432882738, 0.9901578285483587, 0.9987690123126133, 0.996900832488101, 0.9935208255576288, 0.9914284682657839, 0.9921272338897229, 0.9998370056045915, 0.9933446046754804, 0.9946171719127235, 0.9993886131658977, 0.9904020480935403, 0.9976175897609998, 0.990494546626699, 0.9976422557395955, 0.986707398427262, 0.9878082815460304, 0.9999255375019185, 0.9956281203627912, 0.994101109143477, 0.995854385946858, 0.9993147381337829, 0.9988549625429928, 0.9899886473861889, 0.9916577887941642, 0.9929132624942069, 0.9853161155096266]
[3, 7, 12, 18, 19, 20, 23, 26, 28, 29, 32, 36, 37, 52, 53, 56, 60, 63, 65, 77, 81, 82, 86, 91, 93, 94, 95, 97, 100, 107, 110]
31
31
31


In [211]:
unlabel_5 = unlabel_5.loc[[3, 7, 12, 18, 19, 20, 23, 26, 28, 29, 32, 36, 37, 52, 53, 56, 60, 63, 65, 77, 81, 82, 86, 91, 93, 94, 95, 97, 100, 107, 110],:]

In [218]:
frame_5 = [train4_4,unlabel_5]
train4_5 = pd.concat(frame_5)
len(train4_5)
x_train_5 = tfidf_vect.transform(train4_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train4_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6721311475409836
                  precision    recall  f1-score   support

        Analysis       0.42      0.38      0.40        90
      Conclusion       0.66      0.53      0.58        40
           Facts       0.78      0.87      0.82       394
         Invalid       0.47      0.48      0.48       118
           Issue       0.71      0.62      0.67        40
Rule/Law/Holding       0.48      0.28      0.35        50

        accuracy                           0.67       732
       macro avg       0.59      0.53      0.55       732
    weighted avg       0.66      0.67      0.66       732



In [219]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6756238003838771
